## Fig. 2 panel C protein-level concensus annotation

This notebook generates two sets of annotations:
- Graph-based_localization_annotation  
  For each protein, this is the most common annotation in the neighbor annotation 
- consensus_graph_annnotation
  Based on the graph-based localization annotation, for proteins where the graph-based annotation is unclassified, use the cluster annotation


In [37]:
import copy
import os
import random
import sys
from datetime import datetime
from pathlib import Path
import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.io as pio
import umap
import umap.plot
from tqdm.notebook import tqdm

plt.style.use('ggplot')
plt.rcParams['pdf.fonttype'] = 42

script_path = Path.cwd().parent.parent.parent / "script"
data_path = Path.cwd().parent.parent.parent / "data"
sys.path.append(str(script_path))
from external import clustering_workflows
from utils.Jaccard_coefficient import *
from utils.label_processing import attach_annotations

save_path = Path.cwd() / "output"
if not os.path.exists(save_path):
    os.makedirs(save_path)

## load the enrichment table

In [38]:
# %store -r timestamp USE_FROZEN
# if USE_FROZEN:
#     raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
timestamp = "2024-07-24"
print(f"Timestamp: {timestamp}")


Timestamp: 2024-07-24


In [39]:
# define files to load
enrichment_dir = Path.cwd().parent.parent / "enrichment"
enrichment_csv_path = enrichment_dir / "output" / "enrichment_and_volcano_tables" / f'{timestamp}_enrichment_table_NOC_prop.csv'

try:
    # load the file
    enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
except FileNotFoundError:
    print(f"File {enrichment_csv_path} not found.\nPlease run the enrichment analysis first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {enrichment_csv_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [40]:
# check the enrichment metadata columns
# enrichments["metadata"]

In [41]:
# check the sample columns
# enrichments["sample"]

In [20]:
# attach canonical gene names
gene_name_csv = data_path / "external" / "canonical_names_and_Itzhak_data.csv"

lookup_table = pd.read_csv(gene_name_csv)
to_df = enrichments["metadata"].copy()
list_of_cols_to_add = reversed(["Gene_name_canonical"])
for c in list_of_cols_to_add:
    new_col_data = attach_annotations(from_df=lookup_table, to_df=to_df, anno_col=c, from_on="Majority protein IDs", to_on="Majority protein IDs")
    enrichments[("metadata", "Gene_name_canonical")] = new_col_data

# attach ground truth
ground_truth_csv = data_path / "external" / "curated_ground_truth_v9.0.csv"

lookup_table = pd.read_csv(ground_truth_csv)
to_df = enrichments["metadata"].copy()
list_of_cols_to_add = reversed(["compartment"])
for c in list_of_cols_to_add:
    new_col_data = attach_annotations(from_df=lookup_table, to_df=to_df, anno_col=c, from_on="gene_name_canonical", to_on="Gene_name_canonical")
    enrichments[("metadata", "curated_ground_truth_v9.0")] = new_col_data

# attach labels
labels_csv = data_path / "labels" / "cluster_annotation_Dec6.csv"

lookup_table = pd.read_csv(labels_csv)
to_df = enrichments["metadata"].copy()
list_of_cols_to_add = reversed(["cluster_annotation"])
for c in list_of_cols_to_add:
    new_col_data = attach_annotations(from_df=lookup_table, to_df=to_df, anno_col=c, from_on="Majority protein IDs", to_on="Majority protein IDs")
    enrichments[("metadata", "cluster_annotation")] = new_col_data


In [21]:
# remove 14-3-3 from cluster_annotation
enrichments[("metadata", "cluster_annotation")] = enrichments[("metadata", "cluster_annotation")].copy().replace("14-3-3_scaffold","unclassified")

In [22]:
enrichments[("metadata", "cluster_annotation")].unique()

array(['nucleus', 'Golgi', 'cytosol', 'trans-Golgi', 'early_endosome',
       'stress_granule', 'unclassified', 'centrosome', 'ER',
       'recycling_endosome', 'plasma_membrane', 'actin_cytoskeleton',
       'mitochondrion', 'lysosome', 'translation', 'nucleolus',
       'peroxisome', 'p-body', 'ERGIC', 'proteasome'], dtype=object)

## sample selection

In [23]:
# Sample selection for the Leiden and the UMAP algorithms, NOTE: manual sample removal is NOT in this cell

# there are some superfluous samples in this table as well as WTs
# these dont help too much in separating organelles so not counting them
# we also remove the infected samples here as those shouldn't be used in calculating the reference UMAP
cols = list(enrichments["sample"])
meta_cols = list(enrichments["metadata"])
samples = [
    x for x in cols
    if "WT" not in x
    and "harsh" not in x
    and "unsorted" not in x
    and "Infected" not in x
]

# next, we remove additional samples using bait names
genes = [x.split("-")[1] if "-" in x else x for x in samples]
sample_table = pd.DataFrame()
sample_table["samples"] = samples
sample_table["bait"] = genes

bait_drop_list = ["EXOC2"]  # here we are just removing EXOC2
selected_samples = []
for index, row in sample_table.iterrows():
    if row["bait"] not in bait_drop_list:
        selected_samples.append(row["samples"])

In [24]:
# check the selected samples
print(f"the number of selected samples is {len(selected_samples)}")
print(f"the selected samples are {sorted(selected_samples)}")

the number of selected samples is 67
the selected samples are ['01-CAPRIN1', '02-ATG101', '02-COPE', '02-DCP1A', '02-GOLGA2', '02-RICTOR', '03-HSP90AA', '03-HSPA1B', '03-SEC23A', '05-CAV1', '05-EDC4', '05-NCLN', '06-ATP6V1B2', '06-CCDC47', '06-CSNK2A1', '06-CSNK2A2', '07-AP4B1', '07-CLTA', '07-COG8', '07-RAPTOR', '09-ATG101', '09-EDC4', '09-HSP90AA1', '09-PEX3', '09-PSMB7', '09-TOMM20', '10-AP2B1', '10-RTN4', '10-TOMM20', '10-VPS35', '11-CEP350', '11-EEA1', '11-GPR107', '11-SEC31A', '12-ACTB', '12-G3BP1', '12-LAMP1', '12-PNPLA2', '12-RTN4', '12-SEC61B', '12-TOMM20', '13-GOLGA2', '13-RAB11A', '13-RAB14', '13-RAB1A', '13-RAB7A', '14-COPE', '14-GOLGA2', '14-RAB11A', '14-RAB14', '14-RAB1A', '14-RAB7A', '15-G3BP1', '15-GOLGA2', '15-LAMP1', '15-MAP1LC3B', '15-SEC61B', '15-TOMM20', '17-ATP1B3', '17-CAPRIN1', '17-G3BP1', '17-MAP1LC3B', '17-RPL36', '17-SLC30A2', 'NOC_cytosol', 'NOC_nuclear', 'NOC_organelle']


In [25]:
# manually drop a few samples 
to_drop = ["02-EXOC2","06-ATP6V1B2","06-CSNK2A1", "06-CSNK2A2", "07-AP4B1", '02-RICTOR', "07-RAPTOR", "10-AP2B1", "12-PNPLA2"]  # for example: to_drop = ["09-HSP90AA1", "09-PSMB7"]
selected_samples = [x for x in selected_samples if x not in to_drop]  # update the variable: selected_samples

In [26]:
# check the selected samples after manual sample removal
print(f"the number of selected samples is {len(selected_samples)}")
print(f"the selected samples are {sorted(selected_samples)}")

the number of selected samples is 59
the selected samples are ['01-CAPRIN1', '02-ATG101', '02-COPE', '02-DCP1A', '02-GOLGA2', '03-HSP90AA', '03-HSPA1B', '03-SEC23A', '05-CAV1', '05-EDC4', '05-NCLN', '06-CCDC47', '07-CLTA', '07-COG8', '09-ATG101', '09-EDC4', '09-HSP90AA1', '09-PEX3', '09-PSMB7', '09-TOMM20', '10-RTN4', '10-TOMM20', '10-VPS35', '11-CEP350', '11-EEA1', '11-GPR107', '11-SEC31A', '12-ACTB', '12-G3BP1', '12-LAMP1', '12-RTN4', '12-SEC61B', '12-TOMM20', '13-GOLGA2', '13-RAB11A', '13-RAB14', '13-RAB1A', '13-RAB7A', '14-COPE', '14-GOLGA2', '14-RAB11A', '14-RAB14', '14-RAB1A', '14-RAB7A', '15-G3BP1', '15-GOLGA2', '15-LAMP1', '15-MAP1LC3B', '15-SEC61B', '15-TOMM20', '17-ATP1B3', '17-CAPRIN1', '17-G3BP1', '17-MAP1LC3B', '17-RPL36', '17-SLC30A2', 'NOC_cytosol', 'NOC_nuclear', 'NOC_organelle']


## data preprocessing

In [27]:
# save a copy of the tables for UMAP

umap_table = enrichments.droplevel(0, axis=1)[meta_cols + selected_samples].copy()
# normalization and UMAP algorithm are not compatible with any NaN values, so drop them
umap_table = umap_table.dropna(subset=selected_samples)
quants = umap_table[selected_samples].copy()
print(f"the dimensions of the data table saved for UMAP are {quants.shape}")

quants.to_csv(save_path / f"{timestamp}_quants.csv")
umap_table.to_csv(save_path / f"{timestamp}_umap_table.csv")

the dimensions of the data table saved for UMAP are (8540, 59)


convert data into anndata format 


In [28]:
# generating AnnData
selected = enrichments['sample'][selected_samples].copy()
adata = ad.AnnData(selected, dtype=np.float32)

adata.var_names = selected.columns.to_list()
adata.obs_names = enrichments['metadata']["Protein IDs"].to_list()
adata.obs["Protein IDs"] = enrichments['metadata']["Protein IDs"].to_list()
adata.obs["Majority protein IDs"] = enrichments['metadata']["Majority protein IDs"].to_list()
adata.obs["Gene_name_canonical"] = enrichments['metadata']["Gene_name_canonical"].to_list()
adata.obs["curated_ground_truth_v9.0"] = enrichments['metadata']["curated_ground_truth_v9.0"].to_list()
adata.obs["cluster_annotation"] = enrichments['metadata']["cluster_annotation"].to_list()

adata.write_h5ad(save_path / f"adata_{timestamp}.h5ad")

c:\Users\duo.peng\Anaconda3\envs\orgIP\lib\site-packages\anndata\_core\anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
c:\Users\duo.peng\Anaconda3\envs\orgIP\lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## compute knn graph (and save a copy for later use)

In [29]:
# instantiate a clusteringworkflow class (to use the part of workflow that computes the nearest neighbor graph)
kNN_obj = clustering_workflows.ClusteringWorkflow(adata=copy.deepcopy(adata))
# preprocessing
kNN_obj.preprocess(n_pcs=None)
# compute nearest neighbor graph
kNN_obj.calculate_neighbors(n_pcs=None, n_neighbors=20)
adata = kNN_obj.adata
# save a copy of the adata object that contains the kNN graph
knn_adata_path = save_path / f"adata_kNN_{timestamp}.h5ad"
adata.write(knn_adata_path)

In [30]:
kNN_obj.adata

AnnData object with n_obs × n_vars = 8540 × 59
    obs: 'Protein IDs', 'Majority protein IDs', 'Gene_name_canonical', 'curated_ground_truth_v9.0', 'cluster_annotation'
    var: 'mean', 'std'
    uns: 'neighbors'
    obsp: 'distances', 'connectivities'

## generate protein-level consensus annotation

In [31]:
all_majority_ids = adata.obs["Majority protein IDs"].to_list()
all_genes = adata.obs["Gene_name_canonical"].to_list()

annot_df = pd.DataFrame(
    list(zip(
            adata.obs["Majority protein IDs"].to_list(),
            adata.obs["Gene_name_canonical"].to_list(),
            adata.obs["cluster_annotation"].to_list(),
        )),
    columns=["Majority protein IDs", "Gene_name_canonical", "cluster_annotation"],
)

In [32]:
# iterate over all genes and generate protein-level consensus annotation
Graph_based_loc_annot = []
neighbors = []

for idx, gene in tqdm(enumerate(all_genes), total=len(all_genes)): 
    neighbor_list, neighbor_annot_list = gene_neighbor_annots(gene_name=gene, adata=adata, annot_df=annot_df, gene_name_col="Gene_name_canonical", annot_col="cluster_annotation")
    # get the most common annotation in the neighbor annotation
    most_common_annot = Counter(neighbor_annot_list).most_common(1)[0][0]
    if most_common_annot == "unclassified":
        # get the next common annotation
        most_common_annot = Counter(neighbor_annot_list).most_common(2)[1][0]
    Graph_based_loc_annot.append(most_common_annot)    
    neighbors.append([list(Counter(neighbor_annot_list).items())])

  0%|          | 0/8540 [00:00<?, ?it/s]

In [34]:
# add the consensus annotation to the dataframe
annot_df["Graph-based_localization_annotation"] = Graph_based_loc_annot
annot_df["consensus_graph_annnotation"] = Graph_based_loc_annot  # initialize the column
annot_df["neighbors"] = neighbors

# for proteins where the graph-based annotation is unclassified, use the cluster annotation
mask = annot_df["Graph-based_localization_annotation"] == "unclassified"
print("Number of proteins with unclassified graph-based annotation:", sum(mask))
annot_df.loc[mask, "consensus_graph_annnotation"] = annot_df.loc[mask, "cluster_annotation"]

# save the annotation table
annot_df.to_csv(save_path / f"{timestamp}_graph-based_annotations.csv")

Number of proteins with unclassified graph-based annotation: 1147


In [35]:
# add the annotations to the adata object
adata.obs["Graph-based_localization_annotation"] = annot_df["Graph-based_localization_annotation"].to_list()
adata.obs["consensus_graph_annnotation"] = annot_df["consensus_graph_annnotation"].to_list()
# save a copy of the (updated) adata object, overwriting the previous one
adata_path = save_path / f"adata_kNN_{timestamp}.h5ad"
adata.write(adata_path)

# add the annotations to the umap_table
umap_table.insert(6, "consensus_graph_annnotation", annot_df["consensus_graph_annnotation"].to_list())
umap_table.insert(6, "Graph-based_localization_annotation", annot_df["Graph-based_localization_annotation"].to_list())
umap_table.insert(6, "neighbors", annot_df["neighbors"].to_list())
# save a copy of the (updated) umap_table
umap_table.to_csv(save_path / f"{timestamp}_umap_table.csv")

In [36]:
umap_table

,Protein IDs,Majority protein IDs,Gene names,Gene_name_canonical,curated_ground_truth_v9.0,cluster_annotation,neighbors,Graph-based_localization_annotation,consensus_graph_annnotation,12-LAMP1,...,11-SEC31A,09-HSP90AA1,10-RTN4,09-TOMM20,11-CEP350,09-ATG101,10-TOMM20,NOC_cytosol,NOC_organelle,NOC_nuclear
0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124,A0A023T6R1;Q96A72;F5H6P7;F5H6N1,FLJ10292;MAGOHB,MAGOHB,NaN,nucleus,"[[(nucleus, 16), (ER, 1), (unclassified, 4)]]",nucleus,nucleus,-0.141427,...,0.084513,-1.310187,-0.369807,-0.294506,0.212616,3.018858,-1.547331,0.235748,0.241247,0.523005
1,Q9Y5S9;A0A023T787;A0A0J9YW13,Q9Y5S9;A0A023T787,RBM8A;RBM8,RBM8A,NaN,nucleus,"[[(nucleus, 45), (unclassified, 9), (nucleolus...",nucleus,nucleus,-0.588500,...,0.979850,-2.443700,-1.899050,-2.430950,0.895300,-0.728450,-2.969400,0.362391,0.286889,0.350721
2,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5,TM9SF4,TM9SF4,Golgi,Golgi,"[[(trans-Golgi, 8), (Golgi, 42), (ERGIC, 3)]]",Golgi,Golgi,3.703700,...,1.688300,-5.186684,1.852200,-2.612500,0.397300,2.393000,-2.685600,0.054846,0.654148,0.291006
3,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...,PTEN,PTEN,NaN,cytosol,"[[(cytosol, 20)]]",cytosol,cytosol,0.261350,...,0.361729,-0.195980,0.269211,-0.420635,-0.067346,-0.332397,0.223960,1.000000,0.000000,0.000000
4,Q99805;A0A024QYR8;B3KSG9,Q99805;A0A024QYR8;B3KSG9,TM9SF2,TM9SF2,Golgi,trans-Golgi,"[[(Golgi, 6), (plasma_membrane, 1), (trans-Gol...",trans-Golgi,trans-Golgi,5.499848,...,1.179800,-3.389100,1.524950,-1.385850,-0.438800,-0.412350,-2.382900,0.083591,0.697825,0.218584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8535,X5D7P8,X5D7P8,STK39,STK39,NaN,cytosol,"[[(unclassified, 11), (cytosol, 43)]]",cytosol,cytosol,-0.030088,...,1.326828,-0.147448,0.818616,-0.715761,-0.227490,0.874124,0.660835,0.765637,0.145727,0.088636
8536,X5D8X9,X5D8X9,CNTNAP2,CNTNAP2,NaN,plasma_membrane,"[[(plasma_membrane, 20), (nucleolus, 1)]]",plasma_membrane,plasma_membrane,-0.052453,...,1.567885,-0.780498,-0.554957,-1.183749,1.758580,0.498581,-0.680795,0.000000,0.802964,0.197036
8537,X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...,X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...,CRMP1,CRMP1,NaN,cytosol,"[[(cytosol, 21), (unclassified, 1)]]",cytosol,cytosol,0.327787,...,1.309921,1.010047,-0.362585,1.048421,-0.334151,-0.265872,-0.425032,1.000000,0.000000,0.000000
8538,X5DQZ7,X5DQZ7,GPX1,GPX1,NaN,mitochondrion,"[[(mitochondrion, 30), (unclassified, 5), (cyt...",mitochondrion,mitochondrion,0.261512,...,-1.048860,3.884103,0.371638,2.954144,-0.481100,-0.570653,2.896576,0.720741,0.279259,0.000000
